In [1]:
import pandas as pd
import requests
import json
import os as os
import logging


In [34]:
pathforjsons = "C:/Users/prksh/Downloads/CLusteringTR/Pilot4/json_response"
samples = "C:/Users/prksh/Downloads/CLusteringTR/Pilot4/Samples.txt"

In [35]:
os.chdir(pathforjsons)

In [ ]:

body={
   "email" : "dinesh@4basecare.com",
   "password" :"dinesh@123"
}
token = requests.post("https://www.v2.api.ecrf.4basecare.co.in/user/login", json=body).json()['payLoad']['authToken']
patient = 'IN-423-VBVD'
url = f"https://www.v2.api.ecrf.4basecare.co.in/integration/getExternalApiResponseByPatientId/{patient}"
headers = {'Authorization': f'Bearer {token}'}
rs = requests.get(url, headers = headers).json()
rs

In [5]:

api_url = "https://www.v2.api.ecrf.4basecare.co.in/integration/getExternalApiResponseByPatientId/"
pat_df = pd.read_csv(samples, sep='\t')
# List of patient IDs
patient_ids= pat_df['samples'].to_list()
open('data_export.log', 'a').close()
logging.basicConfig(filename='data_export.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
patient_ids[:5]

['IN-423-WL2AF',
 'IN-423-W9NAC',
 'IN-423-WHHAA',
 'IN-423-XBVAM',
 'IN-423-WL1A2']

In [6]:
def fetch_and_export_data(patient_id, token):
    url = f"{api_url}{patient_id}"
    headers = {'Authorization': f'Bearer {token}'}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        # Parse the JSON response
        patient_data = response.json()
        
        # Export the patient data as a JSON file
        output_file = f"{patient_id}_data.json"
        with open(output_file, 'w') as json_file:
            json.dump(patient_data, json_file, indent=2)
        logging.info(f"Data for {patient_id} exported successfully.")
    else:
        logging.error(f"Error fetching data for {patient_id}. Status code: {response.status_code}")

In [7]:
# Loop through each patient ID and fetch/export data


for p in range(len(patient_ids)):
    fetch_and_export_data(patient_ids[p], token)
    print(f' {p} || {len(patient_ids)}', end="\r")

In [36]:
json_list=os.listdir(os.getcwd())
json_list=[i for i in json_list if i.endswith('.json')]
print(json_list[:5])

['IN-423-UGZB_data.json', 'IN-423-UI3A_data.json', 'IN-423-UJHA_data.json', 'IN-423-UL2H_data.json', 'IN-423-VADB_data.json']


In [9]:
def json_cd(info,path, output_path):
    df = pd.read_json(path)
    df.drop(['success', 'message'], axis=1, inplace=True)
    df=df.T
    df.to_json
    df[info].to_json(output_path, orient='records', indent=2)


In [37]:
info="cancerDetails"
os.makedirs(info, exist_ok=True)
for i in range(len(json_list)):
    json_cd(info,json_list[i], f"{pathforjsons}/{info}/{json_list[i]}_{info}.json")
    print(f' {i} || {len(json_list)}', end="\r")
    
os.chdir(f"{pathforjsons}/{info}/")

In [38]:
cd_list = [file for file in os.listdir(os.getcwd()) if file.endswith('.json')]
cd_list[:5]

['IN-423-UGZB_data.json_cancerDetails.json',
 'IN-423-UI3A_data.json_cancerDetails.json',
 'IN-423-UJHA_data.json_cancerDetails.json',
 'IN-423-UL2H_data.json_cancerDetails.json',
 'IN-423-VADB_data.json_cancerDetails.json']

# Test

In [39]:
from pandas.io.json import json_normalize
# Read the JSON file
with open('c:/Users/prksh/Downloads/CLusteringTR/Clustering_Pilot2/json_response/IN-423-UI3A_data.json') as f:
    json_data = json.load(f)

# Remove the outer square brackets
#json_data = json_data[0][0]

# Use json_normalize to flatten the nested structure
df = pd.json_normalize(json_data, sep='_')
df

,success,message,payLoad_patientInfo_patientId,payLoad_patientInfo_patientUUId,payLoad_patientInfo_bmi,payLoad_patientInfo_preMenopausecycleType,payLoad_patientInfo_preMenopausecycleDuration,payLoad_patientInfo_age,payLoad_patientInfo_gender,payLoad_patientInfo_menstruation,...,payLoad_medicalInfo_isMedicalAllergies,payLoad_medicalInfo_medicalAllergiesSpecify,payLoad_medicalInfo_immunosupression,payLoad_medicalInfo_imunosupressionSpecify,payLoad_medicalInfo_otherFormsOfTobaccoUse,payLoad_medicalInfo_otherFormsOfTobaccoUseSpecify,payLoad_medicalInfo_cancerDetailsComments,payLoad_medicalInfo_dateOfDiagnosis,payLoad_cancerDetails,payLoad_events
0,True,successfully fetched,IN-423-UI3A,5b46aee0-5b3f-4ff1-86ff-8527736c2f6b,26.4,None,None,64,MALE,None,...,True,Penicillin Allergy,Unknown,None,Unknown,None,None,None,"[{'cancerSiteIdcCode': 'C34', 'cancerSite': 'L...","[{'eventType': 'DIAGNOSTIC_TEST', 'eventDate':..."


In [40]:
merged_df=[]
for i in cd_list:
    with open(i) as f:
        json_data = json.load(f)
    # Remove the outer square brackets
    if not json_data or not json_data[0]:
        print("JSON structure is empty or does not contain any fields.")
    else:
     json_data = json_data[0]
     df = pd.json_normalize(json_data, sep='_')
     df['Sample'] = i.replace(f'_data.json_{info}.json', '')
     merged_df.append(df)
    

JSON structure is empty or does not contain any fields.
JSON structure is empty or does not contain any fields.
JSON structure is empty or does not contain any fields.
JSON structure is empty or does not contain any fields.
JSON structure is empty or does not contain any fields.
JSON structure is empty or does not contain any fields.
JSON structure is empty or does not contain any fields.
JSON structure is empty or does not contain any fields.
JSON structure is empty or does not contain any fields.
JSON structure is empty or does not contain any fields.
JSON structure is empty or does not contain any fields.
JSON structure is empty or does not contain any fields.
JSON structure is empty or does not contain any fields.
JSON structure is empty or does not contain any fields.
JSON structure is empty or does not contain any fields.
JSON structure is empty or does not contain any fields.
JSON structure is empty or does not contain any fields.
JSON structure is empty or does not contain any 

In [31]:
json_data


{'eventType': 'DIAGNOSTIC_TEST',
 'eventDate': '2024-03-07',
 'diagnosticEventType': 'MOLECULAR_PATHOLOGY',
 'details': {'ihc': [],
  'fish': [],
  'cytogenetics': [],
  'gene_Expression_Assays': [],
  'pcr': [],
  'sanger': [],
  'selectMethod': 'NGS',
  'impression': 'Clinical Indication - Glioblastoma.',
  'dateOfTest': '2024-03-07',
  'approximateDate': None,
  'ngs': [{'createdOn': None,
    'siteOfLusions': [{'fusion': None,
      'variation_Missense_mutation': None,
      'variation_Inframe_deletion': None,
      'variation_Inframe_insertion': None,
      'variation_Indel': None,
      'variation_Splice_site': None,
      'variation_Frame_shift_mutation': None,
      'variation': None,
      'geneName': 'MDM2',
      'geneMarker': 'CNV',
      'clinicalSignificanceOfTheVariant': 'Pathogenic',
      'drugComments': None,
      'referenceID': None,
      'evidence': ' Level 3',
      'mutantAlleleFraction': 0.0,
      'ngsSelectedDrugs': [{'drugsRecommended': [],
        'trialDru

In [41]:
pd.concat(merged_df).to_csv(f"C:/Users/prksh/Downloads/CLusteringTR/pilot4_details_{info}.csv")